In [1]:
import os

In [2]:
%pwd

'/home/nilanj/PyCharmProjects/ML/Chicken-diesease-classification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/nilanj/PyCharmProjects/ML/Chicken-diesease-classification'

In [5]:
# After upadting config I will update entity as I don't have any secret and I am going to update params later
 # you can create custom return type usin entity

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
      root_dir: Path
      source_URL: str
      local_data_file: Path
      unzip_dir: Path

In [8]:
# now update configuration from config
from cnnClassifire.constants import *
from cnnClassifire.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir)
        return data_ingestion_config 



In [9]:
# 6. Update the components

import os
import urllib.request as request
import zipfile
from cnnClassifire import logger
from cnnClassifire.utils.common import get_size


class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file)
            logger.info(f"{filename} download! with following info:\n{headers}")
        else:logger.info(f"File already exists of size:{get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path:str
        Extract the zip file in to the data directory
        Fucntion returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
# 7. update pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e    


[2023-08-18 13:14:54,779:INFO:common:yaml file: config/config.yaml loaded successfully]
[2023-08-18 13:14:54,782:INFO:common:yaml file: params.yaml loaded successfully]
[2023-08-18 13:14:54,784:INFO:common:created directory at: artifacts]
[2023-08-18 13:14:54,786:INFO:common:created directory at: artifacts/data_ingestion]
[2023-08-18 13:14:54,787:INFO:1485634637:File already exists of size:~ 11345 KB]


<h3> to convert in modular coding let's floow the flow </h3>